In [1]:
import os
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import math
import cv2
import glob
import copy
from skimage.feature import hog
from pathlib import Path
import random
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
import pickle
import librosa

import gradio as gr

In [2]:
def load_model(path):
    with open(path, 'rb') as f:
        clf = pickle.load(f)
        return clf

In [3]:
def get_hog(img_array = None):
  gray_img = cv2.cvtColor(((img_array)*255),cv2.COLOR_RGB2GRAY)
  hog_features, hog_image = hog(gray_img, orientations=8, pixels_per_cell=(4, 4),cells_per_block=(1, 1), visualize=True)
  return hog_features, hog_image

In [4]:
def get_sift(img_array = None, num_features = 10):
  sift_final_des = np.zeros((num_features,128), dtype = np.float64)
  gray_img = cv2.cvtColor(((img_array)*255),cv2.COLOR_RGB2GRAY)
  sift = cv2.SIFT_create()
  sift_kp, sift_des = sift.detectAndCompute(gray_img.astype('uint8'),None)
  if sift_des is None:
    sift_des = np.zeros((num_features,128), dtype = np.float64)
  sift_des = np.nan_to_num(sift_des, nan=0, posinf=0, neginf=0)
  max_des = np.max(sift_des)
  max_des = 1
  if max_des is None or max_des == 0:
    max_des = 1
  sift_des_norm = sift_des/max_des
  sift_final_des[0:min(sift_des_norm.shape[0], num_features) , :] = sift_des_norm[0:min(sift_des_norm.shape[0], num_features) , :]
  return sift_final_des

In [5]:
def get_all_features(img, aud):
    img_file = cv2.imread(img)
    hog_features, hog_image = get_hog(img_array = img_file)
    #print(len(hog_features))
    sift_features = get_sift(img_array = img_file, num_features = 10)
    is0,is1 = sift_features.shape
    sift_features = sift_features.reshape(is0*is1)
    #print(sift_features.shape)
    
    y, sr = librosa.load(aud)
    mfccs = librosa.feature.mfcc(y=y, sr=sr)
    as0,as1 = mfccs.shape
    mfccs = mfccs.reshape(as0*as1)
    #print(mfccs.shape)

    if len(hog_features) > 40000:
        hog_features = hog_features[0:40000]
    else:
        pad_length = 40000 - len(hog_features)
        hog_features = np.pad(hog_features, (0, pad_length), 'constant')

    if len(mfccs) > 300:
        mfccs = mfccs[0:300]
    else:
        pad_length = 300 - len(mfccs)
        mfccs = np.pad(mfccs, (0, pad_length), 'constant')
    
    features = np.concatenate((hog_features, sift_features,mfccs), axis=None).reshape(1,-1)
    return features

In [6]:
aud_probe = '/home/ubuntu/food-101-small/Probe/hamburger/hamburger.wav'
y, sr = librosa.load(aud_probe)

In [7]:
from IPython.display import Audio

Audio(aud_probe, rate=sr)

In [10]:
def search(Audio, Gallery):
    clf = load_model('/home/ubuntu/model.pkl')
    default_img = '/home/ubuntu/food-101-small/Other/No_Image_Available.jpg'
    print("Model Loaded Successfully and Searching....", flush=True)
    gallery_all_list = glob.glob(Gallery + '/*')
    #print(Audio)
    for gitem in gallery_all_list:
        for img in glob.glob(gitem + '/*.jpg'):
            X = get_all_features(img, Audio)
            y_pred = clf.predict(X)
            #print(y_pred)
            if y_pred[0] == 1:
                print("Search Completed - Matching Image Found")
                return img, Audio
            break
    print("Search Completed - No Matching Image Found")
    return default_img, Audio

In [14]:
img, audio = search('/home/ubuntu/food-101-small/Probe/churros/churros.wav', '/home/ubuntu/food-101-small/Gallery')

Model Loaded Successfully and Searching....
Search Completed - Matching Image Found


In [ ]:
'''
Case1 : Match Found

Audio : /home/ubuntu/food-101-small/Probe/churros/churros.wav
Gallery : /home/ubuntu/food-101-small/Gallery

Case2 : No Match Found

Audio : /home/ubuntu/food-101-small/Probe/hamburger/hamburger.wav
Gallery : /home/ubuntu/food-101-small/Gallery

Case 3 : Incorrect Match Found

Audio : /home/ubuntu/food-101-small/Probe/chicken_curry/chicken_curry.wav
Gallery : /home/ubuntu/food-101-small/Gallery
'''

In [16]:
import gradio as gr

demo = gr.Interface(
    title="See with Sound: Image Retrieval Redefined!",
    fn=search,
    inputs=["text", "text"],
    outputs=["image", "audio"],
)
demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://cdaca6f1e552e36e6c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Model Loaded Successfully and Searching....
Search Completed - Matching Image Found
Model Loaded Successfully and Searching....
Search Completed - Matching Image Found
Model Loaded Successfully and Searching....
Search Completed - No Matching Image Found
Model Loaded Successfully and Searching....
Search Completed - Matching Image Found
Model Loaded Successfully and Searching....
Search Completed - No Matching Image Found
Model Loaded Successfully and Searching....
Search Completed - Matching Image Found
Model Loaded Successfully and Searching....
Search Completed - Matching Image Found
